In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_parquet("data/monthly_averages.parquet")

In [ ]:
fig, time_ax = plt.subplots(figsize=(5, 3), dpi=150)

color = 'tab:blue'
time_ax.set_ylabel("avg. monthly trip duration / minutes", color=color)

(df["trip_duration_mean"] / 60).plot(
       ax=time_ax, color=color, label="trip duration"
)
time_ax.tick_params(axis='y', labelcolor=color)


dist_ax = time_ax.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:red'
dist_ax.set_ylabel("avg. monthly trip distance", color=color)
df["trip_distance_mean"].plot(ax=dist_ax, color=color, label="trip distance")
dist_ax.tick_params(axis='y', labelcolor=color)

time_ax.set_xlabel("year")
plt.title("Average monthly trip durations and distances")
fig.set_layout_engine("constrained")
plt.savefig("trip_lenghts_monthly_averages.png")